In [1]:
import pandas as pd
import numpy as np
import random

import quantlib
import quantlib.indicators_cal as indicators_cal
import quantlib.diagnostics_utils as diagnostics_utils

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_instruments_from_df(df):
    instruments = []
    for col in df.columns:
        inst = col.split(" ")[0]
        if "USD" in inst and inst not in instruments:
            instruments.append(inst)
    return instruments

origin_df = pd.read_excel("../crypto_historical_4h.xlsx", engine="openpyxl", index_col='open_time')

In [3]:
"""
In this notebook, we want to generalize the framework to explore the factor effect in crypto trading

Steps to do this task:
1. Read the crypto data, whatever the format, we still might change the format later
2. Filter data at the very beginning could be not appropriate, because there are always listings and unlistings
3. Take the index: datetime as a important thing to track trades
4. We should have a function to calculate factor, for example:
    def get_momentum(origin_df):
        return {token:{
            "score": score,
            "rank": rank
        }}
5. Before making every trade, calculate the sum of scores
6. Choose symbols to trade accordingly

"""
instruments = get_instruments_from_df(origin_df)
instruments

['BTCUSDT',
 'ETHUSDT',
 'BCHUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'LTCUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'LINKUSDT',
 'XLMUSDT',
 'ADAUSDT',
 'XMRUSDT',
 'DASHUSDT',
 'ZECUSDT',
 'XTZUSDT',
 'ATOMUSDT',
 'ONTUSDT',
 'IOTAUSDT',
 'BATUSDT',
 'VETUSDT',
 'NEOUSDT',
 'QTUMUSDT',
 'IOSTUSDT',
 'THETAUSDT',
 'ALGOUSDT',
 'ZILUSDT',
 'KNCUSDT',
 'ZRXUSDT',
 'COMPUSDT',
 'OMGUSDT',
 'DOGEUSDT',
 'SXPUSDT',
 'KAVAUSDT',
 'BANDUSDT',
 'RLCUSDT',
 'WAVESUSDT',
 'MKRUSDT',
 'SNXUSDT',
 'DOTUSDT',
 'DEFIUSDT',
 'YFIUSDT',
 'BALUSDT',
 'CRVUSDT',
 'RUNEUSDT',
 'SUSHIUSDT',
 'SRMUSDT',
 'EGLDUSDT',
 'SOLUSDT',
 'ICXUSDT',
 'STORJUSDT',
 'BLZUSDT',
 'UNIUSDT',
 'AVAXUSDT',
 'FTMUSDT',
 'HNTUSDT',
 'ENJUSDT',
 'FLMUSDT',
 'TOMOUSDT',
 'RENUSDT',
 'KSMUSDT',
 'NEARUSDT',
 'AAVEUSDT',
 'FILUSDT',
 'RSRUSDT',
 'LRCUSDT',
 'MATICUSDT',
 'OCEANUSDT',
 'CVCUSDT',
 'BELUSDT',
 'CTKUSDT',
 'AXSUSDT',
 'ALPHAUSDT',
 'ZENUSDT',
 'SKLUSDT',
 'GRTUSDT',
 '1INCHUSDT',
 'CHZUSDT',
 'SANDUSDT',
 'ANKRUSDT',
 'BTSUSDT',
 'LI

In [4]:
look_back = 50
ignore = 5
symbol = "BTCUSDT"

class MomentumFactor:
    """
    @Params:
        df: the original df contains all symbol information
        index: the index that a trading event should happend
        look_back: look back period to calculate the momentum
        ingore: int, to prevent short term reversing, ignore the most recent klines
    """
    def __init__(self, look_back, ignore):
        self.look_back = look_back
        self.ignore = ignore
        
    def calculate(self, df, symbol):
        close_prices = df["{} close".format(symbol)]
        momentum_value = close_prices.shift(self.ignore) / close_prices.shift(self.look_back+self.ignore) - 1
        return momentum_value


In [5]:
"""
Every time we trigger trade function, we calculate all factors for all symbols
"""

def calculate_factor_scores(df, target_symbols, factors, trade_index):
    score_dict = {}
    for symbol in target_symbols:
        factor_scores = sum([factor.calculate(df, symbol)[trade_index] for factor in factors])
        score_dict[symbol] = factor_scores
    return score_dict

def trade(df, target_symbols, factors, trade_index):
    """
    @Params:
        factors: a list of factors that we want to use
        trade_index: the index that the trade is happening
    """
    score_dict = calculate_factor_scores(df, target_symbols, factors, trade_index)
    print (score_dict)

momentum = MomentumFactor(look_back, ignore)
factors = [momentum]
#trade(origin_df, instruments, factors, -1)
momentum.calculate(origin_df, "OPUSDT")

open_time
2022-07-01 12:00:00         NaN
2022-07-01 16:00:00         NaN
2022-07-01 20:00:00         NaN
2022-07-02 00:00:00         NaN
2022-07-02 04:00:00         NaN
                         ...   
2023-04-23 00:00:00   -0.071299
2023-04-23 04:00:00   -0.123951
2023-04-23 08:00:00   -0.124019
2023-04-23 12:00:00   -0.123948
2023-04-23 16:00:00   -0.081236
Name: OPUSDT close, Length: 1778, dtype: float64

In [18]:
from collections import defaultdict

def get_rankings(factor_dict, n_ranks=5):
    """
    According to factor scores in factor_dict, generate rankings for each factor
    Returns:
        dict(rank: [symbol1, symbol2])
    """
    sorted_symbols = sorted(factor_dict, key=factor_dict.get, reverse=True)

    # Calculate the rank for each symbol
    num_symbols = len(sorted_symbols)
    rank_size = num_symbols // n_ranks + (num_symbols % n_ranks > 0)
    ranks_dict = defaultdict(list)
    for i, symbol in enumerate(sorted_symbols):
        rank = i // rank_size
        ranks_dict[rank].append(symbol)

    return ranks_dict

def is_rebalancing(i, rebalance_period):
    return i!=1 and i%rebalance_period==1

def get_rank_porfolio(df, target_symbols, factor_specs, trade_index):

    rank_pofolio = []
    for i, (factor, n_ranks) in enumerate(factor_specs):
        # Filter available trading tokens
        max_look_back = factor.look_back
        if trade_index - max_look_back <= 0:
            return None
        active_symbols = get_target_symbols(df.iloc[trade_index - max_look_back])
        target_symbols = set(target_symbols).intersection(active_symbols)

        # Calculate scores for every symbol
        factor_dict = {}
        for symbol in target_symbols:
            factor_dict[symbol] = factor.calculate(df, symbol)[trade_index]
        rank_dict = get_rankings(factor_dict, n_ranks)
        rank_pofolio.append(rank_dict)

    return rank_pofolio
    

def insert_record(dt, rank_df, rank_portfolio, factor_specs):
    """
    rank_df: DataFrame with columns for each rank and each instrument, as well as columns for the forward returns of each instrument
    rank_portfolio: list of rank dictionaries, where each rank dictionary maps ranks to a list of symbols

    """
    # Create a new row for the rank details
    new_row = pd.DataFrame(index=[dt])

    # Combine ranks
    rank_nums = [x[1] for x in factor_specs]
    rank_combinations = get_all_rank_combinations(rank_nums)

    for agg_rank in rank_combinations:
        rank_to_string = [str(rank) for rank in agg_rank]
        col_name = f"rank_{'_'.join(rank_to_string)}"
        symbols = select_intersection_symbols(agg_rank, rank_portfolio)
        new_row[col_name] = ','.join(symbols)
    
    # Concatenate the new row to the rank_df DataFrame
    rank_df = pd.concat([rank_df, new_row])
    return rank_df

def select_intersection_symbols(agg_rank, rank_portfolio):
    """Find target intersection symbols for every aggregation rank"""
    symbol_candidates = []
    for i, rank in enumerate(agg_rank):
        symbols = rank_portfolio[i][rank]
        symbol_candidates.append(symbols)

    rank1 = set(symbol_candidates[0])
    target_symbols = rank1.intersection(*symbol_candidates[1:])
    return target_symbols

def get_all_rank_combinations(rank_nums):
    if not rank_nums:
        return []
    result = [[]]
    for n in rank_nums:
        new_result = []
        for i in range(n):
            for r in result:
                new_result.append(r + [i])
        result = new_result
    return result

def analyse_factor_rank():
    pass

def trade_with_ranks():
    pass

def get_target_symbols(row):
    target_symbols = []
    for inst in instruments:
        if row[f"{inst} active"] == True:
            target_symbols.append(inst)

    return target_symbols

def _add_fwd_ret(df, period):
    all_symbols = get_instruments_from_df(df)
    for symbol in all_symbols:
        df["{} {} fwd ret".format(symbol, period)] = df["{} close".format(symbol)].shift(-period) / df["{} close".format(symbol)] - 1

    return df

def lookup_fwd_ret_for_symbols(df, dt, rank_symbols, rebalance_period, weights=None):
    """For specific dt and symbols, take the average of forward returns"""
    rank_symbols = rank_symbols.split(',')
    total_fwd_ret = 0
    for symbol in rank_symbols:
        fwd_ret = df.loc[dt, f"{symbol} {rebalance_period} fwd ret"]
        total_fwd_ret += fwd_ret

    avg_fwd_ret = round(total_fwd_ret / len(rank_symbols), 4)
    return avg_fwd_ret
    
    

def lookup_fwd_ret(df, rank_df, rebalance_period, weights=None):
    """Combine forward returns for the rank dataframe"""
    rank_cols = rank_df.columns
    for dt in rank_df.index:
        for ranks in rank_cols:
            rank_symbols = rank_df.loc[dt, ranks]
            fwd_ret = lookup_fwd_ret_for_symbols(df, dt, rank_symbols, rebalance_period, weights)
            rank_df.loc[dt, f"{ranks} fwd_ret"] = fwd_ret

    return rank_df
            
    

def backtest(df, rebalance_period, factor_specs):
    # rank_df, every row should contains the rank details for that time
    
    df = _add_fwd_ret(df, rebalance_period)
    rank_df = pd.DataFrame()

    for i, dt in enumerate(df.index):
        if is_rebalancing(i, rebalance_period):
            target_symbols = get_target_symbols(df.iloc[i])
            rank_porfolio = get_rank_porfolio(df, target_symbols, factor_specs, trade_index=i)
            if not rank_porfolio:
                continue
            rank_df = insert_record(dt, rank_df, rank_porfolio, factor_specs)

    rank_df = lookup_fwd_ret(df, rank_df, rebalance_period, weights=None)
            

    return rank_df
    #trade_with_ranks(rank_df)

momentum1 = MomentumFactor(look_back, ignore)
momentum2 = MomentumFactor(look_back=20, ignore=2)
factor_specs = [(momentum1, 5), (momentum2, 2)]
backtest(origin_df, rebalance_period=15, factor_specs=factor_specs)
#get_all_rank_combinations([5,5,5,5])

,rank_0_0,rank_1_0,rank_2_0,rank_3_0,rank_4_0,rank_0_1,rank_1_1,rank_2_1,rank_3_1,rank_4_1,rank_0_0 fwd_ret,rank_1_0 fwd_ret,rank_2_0 fwd_ret,rank_3_0 fwd_ret,rank_4_0 fwd_ret,rank_0_1 fwd_ret,rank_1_1 fwd_ret,rank_2_1 fwd_ret,rank_3_1 fwd_ret,rank_4_1 fwd_ret
2022-07-11 16:00:00,"ZENUSDT,COMPUSDT,ARUSDT,SRMUSDT,RENUSDT,JASMYU...","XMRUSDT,DEFIUSDT,IOTAUSDT,DENTUSDT,STMXUSDT,AN...","MKRUSDT,DASHUSDT,QTUMUSDT,SXPUSDT,LTCUSDT,TRXU...","DARUSDT,ETCUSDT,XEMUSDT,HNTUSDT,XRPUSDT,ALGOUS...","BNXUSDT,KAVAUSDT,MASKUSDT,LINKUSDT,DOTUSDT,BTC...","IOTXUSDT,LINAUSDT,CRVUSDT,GMTUSDT,CELRUSDT,BTS...","API3USDT,ALPHAUSDT,ENSUSDT,SOLUSDT,ENJUSDT,YFI...","ZRXUSDT,KNCUSDT,AXSUSDT,C98USDT,APEUSDT,REEFUS...","CHRUSDT,FILUSDT,ALICEUSDT,OPUSDT,MTLUSDT,CTSIU...","CTKUSDT,DOGEUSDT,XLMUSDT,HOTUSDT,BELUSDT,OGNUS...",-0.0274,-0.0514,-0.0487,-0.0322,-0.0467,-0.0171,-0.0268,-0.0407,-0.0520,-0.0415
2022-07-14 04:00:00,"BNXUSDT,NEOUSDT,AVAXUSDT,ONEUSDT,SRMUSDT,DEFIU...","CTKUSDT,LINKUSDT,ARUSDT,XTZUSDT,ALGOUSDT,FILUS...","DARUSDT,ETCUSDT,XLMUSDT,IOSTUSDT,HNTUSDT,KLAYU...","HOTUSDT,ALICEUSDT,ZILUSDT,IMXUSDT,YFIUSDT,THET...","APEUSDT,SNXUSDT,ATOMUSDT,WAVESUSDT,ANCBUSD,RSR...","ZENUSDT,COMPUSDT,JASMYUSDT,IOTXUSDT,TOMOUSDT,U...","DASHUSDT,PEOPLEUSDT,DENTUSDT,IOTAUSDT,LINAUSDT...","QTUMUSDT,ZRXUSDT,ONTUSDT,DOTUSDT,MTLUSDT,STORJ...","KAVAUSDT,MKRUSDT,MASKUSDT,C98USDT,ENSUSDT,ATAU...","DOGEUSDT,AXSUSDT,SXPUSDT,API3USDT,BELUSDT,GALU...",0.1199,0.0988,0.0867,0.0704,0.0658,0.0971,0.0821,0.0933,0.1059,0.0923
2022-07-16 16:00:00,"COMPUSDT,ARUSDT,ALGOUSDT,RENUSDT,DEFIUSDT,SRMU...","NEOUSDT,CTSIUSDT,SOLUSDT,ATOMUSDT,AVAXUSDT,BAN...","ALPHAUSDT,SNXUSDT,MTLUSDT,BCHUSDT,GRTUSDT,YFIU...","ZRXUSDT,GTCUSDT,SANDUSDT,RUNEUSDT,WOOUSDT,RSRUSDT","MKRUSDT,ENSUSDT,STORJUSDT,RLCUSDT,IMXUSDT",ONEUSDT,"BNXUSDT,XTZUSDT,IOTXUSDT,BTCUSDT,TRXUSDT,NEARU...","ETCUSDT,DASHUSDT,LINKUSDT,IOSTUSDT,XLMUSDT,IOT...","QTUMUSDT,MASKUSDT,SXPUSDT,HOTUSDT,ARPAUSDT,HNT...","CTKUSDT,JASMYUSDT,LRCUSDT,ENJUSDT,API3USDT,ALI...",0.0283,0.0601,0.0700,0.0678,0.0420,0.0975,0.0562,0.1106,0.0777,0.0967
2022-07-19 04:00:00,"ETCUSDT,ARUSDT,APEUSDT,OPUSDT,ENSUSDT,CRVUSDT,...","NEOUSDT,SOLUSDT,BCHUSDT,GRTUSDT,DEFIUSDT,GMTUS...","MKRUSDT,DASHUSDT,QTUMUSDT,XEMUSDT,ZENUSDT,FILU...","MASKUSDT,IOTXUSDT,LINAUSDT,GTCUSDT,ENJUSDT,THE...","PEOPLEUSDT,ZRXUSDT,C98USDT,SFPUSDT,BANDUSDT","EGLDUSDT,XMRUSDT,ALGOUSDT,SUSHIUSDT,AAVEUSDT,U...","SRMUSDT,REEFUSDT,CTSIUSDT,BAKEUSDT,COTIUSDT","DARUSDT,KNCUSDT,COMPUSDT,XTZUSDT,BTCUSDT,STMXU...","BNXUSDT,HOTUSDT,RLCUSDT,HNTUSDT,ZILUSDT,ALICEU...","KAVAUSDT,CTKUSDT,DOGEUSDT,SXPUSDT,JASMYUSDT,BT...",0.0517,0.0117,-0.0002,0.0086,0.0162,0.0083,0.0250,0.0109,0.0302,-0.0087
2022-07-21 16:00:00,"SOLUSDT,FTMUSDT,ENJUSDT,FLOWUSDT,RUNEUSDT,AVAX...","AXSUSDT,XMRUSDT,BTCUSDT,KLAYUSDT,GALUSDT,ALICE...","DOGEUSDT,DEFIUSDT,DOTUSDT,GMTUSDT,HBARUSDT,YFI...","GALAUSDT,LINKUSDT,ZRXUSDT,XLMUSDT,HNTUSDT,SXPU...","BNXUSDT,PEOPLEUSDT,BTCDOMUSDT","AAVEUSDT,RVNUSDT,BTSUSDT,ARUSDT","COMPUSDT,LTCUSDT,NEOUSDT,FTTUSDT,CELRUSDT,UNIU...","DASHUSDT,QTUMUSDT,KNCUSDT,ALGOUSDT,TOMOUSDT,ST...","DARUSDT,KAVAUSDT,MKRUSDT,ZENUSDT,HOTUSDT,ANTUS...","CTKUSDT,MASKUSDT,JASMYUSDT,RLCUSDT,ARPAUSDT,OG...",-0.0087,-0.0037,0.0001,0.0056,0.0221,-0.0334,-0.0088,-0.0083,-0.0009,0.0041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-12 16:00:00,"JASMYUSDT,CTSIUSDT,SOLUSDT,FTMUSDT,RUNEUSDT,ON...","TOMOUSDT,RLCUSDT,ARPAUSDT,YFIUSDT,CVXUSDT,BAND...","ICPUSDT,GALAUSDT,DARUSDT,QTUMUSDT,API3USDT,SUS...","XMRUSDT,HOTUSDT,HOOKUSDT,LTCUSDT,HIGHUSDT,IOTX...","LINKUSDT,IDUSDT,AGIXUSDT,ENSUSDT,JOEUSDT,CELRU...","DOGEUSDT,STGUSDT,KNCUSDT,FOOTBALLUSDT,KLAYUSDT...","TLMUSDT,IOTAUSDT,STMXUSDT,REEFUSDT,LUNA2USDT,M...","PEOPLEUSDT,XEMUSDT,AXSUSDT,ARUSDT,GMXUSDT,OPUS...","CTKUSDT,OMGUSDT,ZRXUSDT,ZENUSDT,COMPUSDT,XRPUS...","BNXUSDT,MASKUSDT,SXPUSDT,LRCUSDT,DEFIUSDT,SSVU...",0.0719,0.0496,0.0751,0.0691,0.1574,0.0766,0.0635,0.0706,0.0563,0.1048
2023-04-15 04:00:00,"ICPUSDT,JASMYUSDT,LRCUSDT,SOLUSDT,CVXUSDT,INJU...","MINAUS